In [1]:
import gc
import json
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm

from utils import loadpkl, removeCorrelatedVariables, removeMissingVariables

%matplotlib inline

In [28]:
files = sorted(glob('../features/*.feather'))
df = pd.concat([pd.read_feather(f) for f in tqdm(files, mininterval=60)], axis=1)

# set card_id as index
df.set_index('card_id', inplace=True)
df.head()

  0%|          | 0/1079 [00:00<?, ?it/s]/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pandas/io/feather_format.py:124: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  nthreads=int_use_threads)
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
100%|██████████| 1079/1079 [00:01<00:00, 558.94it/s]


,AOV,AOV_ratio,CLV,CLV_month_diff,CLV_month_diff_ratio,CLV_ratio,Mothers_Day_2018,amount_month_ratio_max,amount_month_ratio_mean,amount_month_ratio_min,...,weekofyear_9,year_0,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,182.318395,1.661881,4.690294e+06,1.705895e+08,-0.008053,0.013005,49.492977,NaN,NaN,-350.00,...,0,0,0,0,0,0,0,0,1,0
C_ID_3d0044924f,101.445060,0.159410,1.071890e+07,1.005798e+07,0.000050,0.000047,56.833333,NaN,NaN,7.99,...,0,0,0,0,0,0,0,0,1,0
C_ID_d639edf6cd,76.651163,0.679063,8.444000e+04,NaN,NaN,0.000367,15.720930,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
C_ID_186d6a6901,118.311688,0.856153,3.805900e+05,NaN,NaN,0.007076,55.623377,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
C_ID_cdbd2c0db2,249.208020,1.067407,2.299008e+06,-9.753155e+07,0.031752,0.078204,61.856099,NaN,NaN,-156.09,...,0,0,0,0,0,0,0,0,1,0


In [30]:
# https://www.kaggle.com/raddar/card-id-loyalty-different-points-in-time
df['hist_observation_date']= df.apply(lambda x: pd.to_datetime(x['hist_purchase_date_max']*10**9)-pd.DateOffset(months=x['hist_month_lag_max']), axis=1)
df['new_observation_date']=df.apply(lambda x: np.nan if np.isnan(x['new_month_lag_min']) else pd.to_datetime(x['new_purchase_date_min']*10**9)-pd.DateOffset(months=x['new_month_lag_min']-1), axis=1)

df['hist_observation_date'] = df['hist_observation_date'].dt.to_period('M').dt.to_timestamp()+pd.DateOffset(months=1)
df['new_observation_date'] = df['new_observation_date'].dt.to_period('M').dt.to_timestamp()

In [29]:
df['hist_purchase_date_max']

card_id
C_ID_92a2005557    1.519551e+09
C_ID_3d0044924f    1.517438e+09
C_ID_d639edf6cd    1.519759e+09
C_ID_186d6a6901    1.519818e+09
C_ID_cdbd2c0db2    1.519850e+09
C_ID_0894217f2f    1.501343e+09
C_ID_7e63323c00    1.519402e+09
C_ID_dfa21fc124    1.513885e+09
C_ID_fe0fdac8ea    1.512825e+09
C_ID_bf62c0b49d    1.519837e+09
C_ID_92853cdb2c    1.519369e+09
C_ID_269d816788    1.514577e+09
C_ID_61d50d7057    1.519021e+09
C_ID_4e07413433    1.508012e+09
C_ID_b6302b31c6    1.509404e+09
C_ID_3b69154173    1.519480e+09
C_ID_9feec11e78    1.519798e+09
C_ID_f6658dbefe    1.517915e+09
C_ID_4a7dda0f9e    1.517765e+09
C_ID_6adae2a906    1.519497e+09
C_ID_0b70ca7347    1.519577e+09
C_ID_25b2509282    1.519713e+09
C_ID_82fb11db22    1.517829e+09
C_ID_e03db05bde    1.519836e+09
C_ID_dcb7c76747    1.519808e+09
C_ID_749cfc0c3a    1.517573e+09
C_ID_4a97126360    1.519820e+09
C_ID_17dea8d446    1.518810e+09
C_ID_fcef0dfd67    1.519800e+09
C_ID_82868156f0    1.496050e+09
                       ...     


In [ ]:
np.isnu